## Amazon Fine Food Reviews
#### Project By:
Girija Suresh Dhawale | Yash Kumar Abinash | Zhenghui Li

### Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews
This dataset consists of reviews of fine foods from Amazon. Reviews include product and user information, ratings, and a plain text review. 

This dataset has a Reviews.csv file and a database.sqlite file which can used to implement sql commands. We have used the sql file in our project by importing the library sqlite3.

##### Data includes:

Timespan: Reviews from Oct 1999 - Oct 2012 

Number of reviews: 568,454 reviews 

Number of users: 256,059 users 

Number of products: 74,258 products 

Number of columns: 10 

Number of users with > 50 reviews: 260

##### Attributes Information:

Id: Row Id

ProductId: Unique identifier for the product

UserId: Unqiue identifier for the user

ProfileName: Profile name of the user

HelpfulnessNumerator: Number of users who found the review helpful

HelpfulnessDenominator: Number of users who indicated whether they found the review helpful or not <br>
Sum of number of users who found the review helpful and who did not find it helpful

Score: Rating between 1 and 5

Time: Timestamp at which the user reviewed the product

Summary: Brief summary of the review

Text: Text of the review

### Objective:
To find out manually if the customers are are liking the product or not is quite time consuming, hence there is a need to build a model that can categorize these reviews to learn about customer feedback.

In this project, we are aiming to determine whether the review is Positive(rating 4 or 5) or Negative (rating 1 or 2) based on the reviews given by a customer.

### Approach:
To determine if a review is positive or negative -

We will use Score/ Rating. A rating that is either 4 or 5 can be considered as positive review. While, rating of 1 or 2 can be considered as negative review. Whereas, if the rating is 3 it means it is neither positive nor negative and thus it can be categorized as neutral review and such reviews are ignored from the analysis.

This is an approximate way of determing the polarity (positivity/ negativity) of a review.

### Step 1: Importing Libraries

In [5]:
import sqlite3
import re
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

%matplotlib inline

In [6]:
%pwd

'C:\\Users\\Girija Dhawale\\FinalProject'

### Step 2: Reading in data 

In [7]:
#connection object to fetch the data from the tables

connectionObj = sqlite3.connect('database.sqlite') 


In [9]:
#discard the reviews with score 3 as it will be neutral, retain the remaining
#As score 3 will not help in deciding whether a review is positive or negative
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", connectionObj) 


In [10]:
#function to replace score greater than 3 by positive and score less than 3 by negative 
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#mapping 
actualScore = filtered_data['Score']
posNegative = actualScore.map(partition) 
filtered_data['Score'] = posNegative

In [11]:
#checking number of attributes and size of the data
filtered_data.shape 
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Step 3: Exploratory Data Analysis

The user with user ID AR5J8UI46CURR has multiple reviews with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text.

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [14]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", connectionObj)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [15]:
#sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [17]:
#drop duplicate data entries, keeping only first occurence of the duplicates
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

Here, in this case the HelpfulnessNumerator is greater than HelpfulnessDenominator which is practically not possible.
Helpfulness Numerator can never be greater than the HelpfulnessDenominator. <br> Because the HelpfulnessDenominator is the sum of number of users who found the review helpful and who did not find it helpful, so it will always be greater.<br>
Thus, we will eliminate such values from calculation.

In [18]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", connectionObj)
display


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [19]:
#retain only those columns where HelpfulnessNumerator is less than or equal to HelpfulnessDenominator
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [20]:
#number of entries
print(final.shape)

#number of positive and negative reviews dataset
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

### Step 4: Data Preprocessing

- Remove the html tags <br>
- Remove any punctuations or set of special characters like , or . or # etc. <br>
- Check if the word is made up of english letters and is not alpha-numeric <br>
- Convert the word to lowercase <br>
- Remove Stopwords <br>
- Snowball Stemming the word <br>

This cleaned text column is used to describe the positive and negative reviews.

In [21]:
#determine sentences containing HTML tags
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;    
    

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


##### Stemming  - 
It means converting a word to its base word. Like, tasty and tasteful both have tast as the stem word. 
##### Stopwords - 
Commonly used words in english. We will loop through the reviews and only store the words which are not in the stopwords.

In [24]:
stop = set(stopwords.words('english')) #set of stopwords
sbs = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

#function to eliminate the word of any html tags
def removehtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

 #function to clean the word of any punctuation or special characters
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sbs.stem('tasty'))

{'as', 'if', 'yourselves', 'i', "don't", 've', 'wasn', 'am', 'his', 'you', 'during', 'shouldn', 'ourselves', "you've", 'or', "couldn't", "shouldn't", 'did', 'doing', 'won', 'on', 'themselves', 'have', 'them', 'weren', "should've", 'off', 'being', 'we', "wouldn't", 'over', 'such', 'how', 'then', "shan't", 'be', 'from', 'who', 't', 'between', 're', 'itself', 'hasn', 'too', "that'll", 'aren', 'most', 'herself', 'mustn', 'it', 'he', 'yours', 'into', 'some', 'once', 'needn', 'before', 'her', "wasn't", 'they', 'couldn', 'himself', "weren't", "needn't", "hasn't", 'been', "aren't", 'ma', 'where', 'but', 'only', 'd', "she's", 'about', 'him', 'further', 'doesn', 'just', 'more', 'than', 'that', 'hadn', 'ours', 'do', 'yourself', 'no', 'there', 'all', 'didn', "didn't", 'these', 'shan', 'does', 'its', 'had', "isn't", 'our', 'which', 'your', 'has', 'at', 'those', 'an', 'own', "it's", 'below', 'wouldn', 'not', 'theirs', 'out', "haven't", "mustn't", 'again', 'hers', 'so', 'same', "mightn't", 'because',

In [26]:
#the code in the following cell takes time as it needs to run on 500k sentences

In [25]:
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from positive reviews here
all_negative_words=[] # store words from negative reviews here
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=removehtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sbs.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews 
                else:
                    continue
            else:
                continue 
   
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("**************")
    
    final_string.append(str1)
    i+=1

In [27]:
#add column of CleanedText that displays data after pre-processing of the review 
final['CleanedText']=final_string 

In [28]:
#CleanedText Column 
final.head(3) 

# store final table into an SQlLite table for future
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

### Bag of Words Approach 

Total number of unique words is the total dimensions we have. Consider it as a big matrix where each word is an index, where each occurence of the word will have value 1. Since we have removed the stopwords, lammetized and stemmed the data entries we will get a matrix with a lower dimension.

In [34]:
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)

In [35]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [36]:
final_counts.get_shape()

(364171, 115281)

### Bi-grams and n-Grams

In [37]:
#get frequency distribution of words
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(20))
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))

Most Common Positive Words :  [(b'like', 139429), (b'tast', 129047), (b'good', 112766), (b'flavor', 109624), (b'love', 107357), (b'use', 103888), (b'great', 103870), (b'one', 96726), (b'product', 91033), (b'tri', 86791), (b'tea', 83888), (b'coffe', 78814), (b'make', 75107), (b'get', 72125), (b'food', 64802), (b'would', 55568), (b'time', 55264), (b'buy', 54198), (b'realli', 52715), (b'eat', 52004)]
Most Common Negative Words :  [(b'tast', 34585), (b'like', 32330), (b'product', 28218), (b'one', 20569), (b'flavor', 19575), (b'would', 17972), (b'tri', 17753), (b'use', 15302), (b'good', 15041), (b'coffe', 14716), (b'get', 13786), (b'buy', 13752), (b'order', 12871), (b'food', 12754), (b'dont', 11877), (b'tea', 11665), (b'even', 11085), (b'box', 10844), (b'amazon', 10073), (b'make', 9840)]


In [38]:
#bi-gram, tri-gram and n-gram
#removing stop words like "not" should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,2) ) #in scikit-learn
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [27]:
final_bigram_counts.get_shape()

(364171, 2910192)

### TF-IDF - Term Frequency-Inverse Document Frequency

It is a combination of two techniques tha is Term Frequency and Inverse Document Frequency.<br>
In TF we take the number of times the word occurs in the sentence divided by total number of words in the sentence. <br>
Whereas, IDF is log of total number of documents divided by number of documents where the words are occuring. 

We get a sparse matrix after this.

In [39]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [40]:
final_tf_idf.get_shape()

(364171, 2910192)

In [41]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [43]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [44]:
# covnert a row in sparsematrix to a numpy array
print(final_tf_idf[3,:].toarray()[0]) 

[0. 0. 0. ... 0. 0. 0.]


In [46]:
# source: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [47]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074
